# Import

In [2]:
import pandas as pd
import re
from tqdm import tqdm
import emoji

tqdm.pandas()
ear = pd.read_csv('피부귀.csv').iloc[:,1:]
vet = pd.read_csv('수의사답변.csv')
vet['images'] = vet['images'].apply(lambda x : x.split('?')[0])
df = pd.read_csv('df.csv')
df['images'] = df['images'].apply(lambda x : x.split('?')[0])

In [3]:
## 데이터프레임 몇 자까지 볼 건지 결정하는 코드
pd.set_option('display.max_colwidth', 500)

## 1. Preprocessing (Lifet)

In [4]:
## 질문, 답변 중 특정 키워드의 문장 제거
ear['question'] = ear['question'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '감사' not in sentence]))
ear['question'] = ear['question'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '안녕' not in sentence]))
ear['question'] = ear['question'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '네!!' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '종료' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '감사' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '궁금' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '라이펫' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if ':)' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '그럼요!' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '몽이가' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '네 ' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '미용사에게 맡기는 것을' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '그럴 가능성이 높아보' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '다행입니다!' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '그렇다면 별도의' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '사진이 있으실까요' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '맞습니다' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '후 그에 대한 치료가 필요' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '대부분은 다시 자랍' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '사진 보내주시면' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '네! 종류가 그렇게 ' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '털빠짐, 각질, 가려움 증상이 있나요' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '사진 첨부해주시면' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '사진을 보내주시면' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '다른 증상은 없을가요?' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '또 문의주세요' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '새로운 장소를 방문한' not in sentence]))
ear['answer'] = ear['answer'].apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if '다른 부위에는 ' not in sentence]))

## 해외거주자 quesion 삭제
ear = ear[~ear.question.str.contains('해외')]
ear = ear[~ear.answer.str.contains('https')]
ear = ear[~ear.answer.str.contains('ACTH')]

## 이모지 제거 및 오타 교정
ear['question'] = ear['question'].apply(lambda x: emoji.replace_emoji(x, replace=''))
ear['answer'] = ear['answer'].apply(lambda x: emoji.replace_emoji(x, replace=''))
ear['answer'] = ear['answer'].apply(lambda x: x.replace('핑요합니다', '필요합니다')) 
ear['answer'] = ear['answer'].apply(lambda x: x.replace('어려루나', '어려우나'))

## 줄바꿈 문자, ㅠㅜ 등 감정표현 제거
ear['question'] = ear['question'].apply(lambda x : x.replace('\n', ' ').replace('ㅠ', '').replace('ㅜ',''))
ear['answer'] = ear['answer'].apply(lambda x : x.replace('\n', ' ').replace('ㅠ', '').replace('ㅜ',''))

## 강아지 이름 제거
ear['answer'] = ear['answer'].apply(lambda x : x.replace('레오가', ''))
ear['answer'] = ear['answer'].apply(lambda x : x.replace('쫑아의', ''))
ear['answer'] = ear['answer'].apply(lambda x : x.replace('조이의', ''))
ear['answer'] = ear['answer'].apply(lambda x : x.replace('헬리오의', ''))

## 공백 관련 전처리
ear['answer'] = ear['answer'].str.strip() ## 양쪽 공백 제거
ear['answer'] = ear['answer'].apply(lambda x : x.replace('.', '. ') )## 띄어쓰기가 안된 경우 띄어쓰기
ear['answer'] = ear['answer'].str.replace(r'\s{2,}', ' ', regex=True) ## 공백 두개 이상인 경우 하나로 줄이기

## 2. Preprocessing ( Dog 지식in )

#### 1) 행 제거

In [5]:
## 이미지 url이 없는 경우 제거
vet = vet.loc[vet.images != 'none']

## 고양이, 토끼, 메추리도 있더라
vet = vet[~vet.title.str.contains('토끼') & ~vet.question.str.contains('토끼') & ~vet.answer.str.contains('토끼')]
vet = vet[~vet.title.str.contains('참새') & ~vet.question.str.contains('참새') & ~vet.answer.str.contains('참새')]
vet = vet[~vet.title.str.contains('조류') & ~vet.question.str.contains('조류') & ~vet.answer.str.contains('조류')]
vet = vet[~vet.title.str.contains('고양이') & ~vet.question.str.contains('고양이') & ~vet.answer.str.contains('고양이')]
vet = vet[~vet.title.str.contains('메추리') & ~vet.question.str.contains('메추리') & ~vet.answer.str.contains('메추리')]
vet = vet[~vet.title.str.contains('앵무새') & ~vet.question.str.contains('앵무새') & ~vet.answer.str.contains('앵무새')]
vet = vet[~vet.title.str.contains('비둘기') & ~vet.question.str.contains('비둘기') & ~vet.answer.str.contains('비둘기')]
vet = vet[~vet.title.str.contains('햄스터') & ~vet.question.str.contains('햄스터') & ~vet.answer.str.contains('햄스터')]
vet = vet[~vet.title.str.contains('비둘기') & ~vet.question.str.contains('비둘기') & ~vet.answer.str.contains('비둘기')]
vet = vet[~vet.title.str.contains('오소리') & ~vet.question.str.contains('오소리') & ~vet.answer.str.contains('오소리')]
vet = vet[~vet.title.str.contains('고슴도치') & ~vet.question.str.contains('고슴도치') & ~vet.answer.str.contains('고슴도치')]
vet = vet[~vet.title.str.contains('딱따구리') & ~vet.question.str.contains('딱따구리') & ~vet.answer.str.contains('딱따구리')]
vet = vet[~vet.title.str.contains('직박구리') & ~vet.question.str.contains('직박구리') & ~vet.answer.str.contains('직박구리')]
vet = vet[~vet.title.str.contains('기니피그') & ~vet.question.str.contains('기니피그') & ~vet.answer.str.contains('기니피그')]

## 수의사, 메디컬센터, 의료센터, 원장 등 전문가의 의견만 남기기
vet = vet[vet.answer.str.contains('수의사') | vet.answer.str.contains('메디컬센터') | vet.answer.str.contains('의료센터') | vet.answer.str.contains('원장')]

## 혈변, 설사, 구토, 슬개골 탈구에 관련된 글 제거
vet = vet[~vet.title.str.contains('혈변') & ~vet.title.str.contains('설사') & ~vet.title.str.contains('구토') & ~vet.title.str.contains('슬개골') & ~vet.title.str.contains('눈물') & ~vet.title.str.contains('점액변') &
    ~vet.title.str.contains('배변') & ~vet.title.str.contains('수의과') & ~vet.title.str.contains('마운팅') & ~vet.title.str.contains('골절') & ~vet.question.str.contains('혈변') & ~vet.question.str.contains('설사') &
    ~vet.question.str.contains('구토') & ~vet.question.str.contains('슬개골') & ~vet.question.str.contains('눈물') & ~vet.question.str.contains('점액변') & ~vet.question.str.contains('배변') & ~vet.question.str.contains('수의과') &
    ~vet.question.str.contains('마운팅') & ~vet.question.str.contains('골절') & ~vet.answer.str.contains('혈변') & ~vet.answer.str.contains('설사') & ~vet.answer.str.contains('구토') & ~vet.answer.str.contains('슬개골') &
    ~vet.answer.str.contains('눈물') & ~vet.answer.str.contains('점액변') & ~vet.answer.str.contains('배변') & ~vet.answer.str.contains('수의과') & ~vet.answer.str.contains('마운팅') & ~vet.answer.str.contains('골절')]

## 성별, 종을 물어보는 경우 제거
vet = vet[~vet.question.str.contains('부정교합')]
vet = vet[~vet.question.str.contains('성별')]
vet = vet[~vet.question.str.contains('무슨 종')]
vet = vet[~vet.answer.str.contains('품종')]
vet = vet[~vet.answer.str.contains('수컷')]
vet = vet[~vet.answer.str.contains('암컷')]
vet = vet[~vet.answer.str.contains('깃털')]
vet = vet[~vet.answer.str.contains('홍역 키트')]
vet = vet[~vet.answer.str.contains('항원검사')]
vet = vet[~vet.answer.str.contains('사람치아로 보입니다')]
vet = vet[~vet.answer.str.contains('여쭤봐도 될까요?')]
vet = vet[~vet.answer.str.contains('axis')]

## 답변에 '오손도손' 이 있는 경우에 강아지 품종 물어보는 글이 많았음
vet = vet[~vet.answer.str.contains('오손도손')]
vet = vet[~vet.answer.str.contains('http:')]

#### 2) 답변 미세 조정

In [6]:
vet.answer = vet.answer.apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if "지식iN" not in sentence]))
vet.answer = vet.answer.apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if "안녕하세요" not in sentence]))
vet.answer = vet.answer.apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if "감사합니다" not in sentence]))
vet.answer = vet.answer.apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if "안녕히 계세요" not in sentence]))
vet.answer = vet.answer.apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if "정상적인 소견이니" not in sentence]))
vet.answer = vet.answer.apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if "질문을 너무 늦게 봤네요" not in sentence]))
vet.answer = vet.answer.apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if "은평동물병원장" not in sentence]))
vet.answer = vet.answer.apply(lambda x: '\n'.join([sentence for sentence in x.split('\n') if "https" not in sentence]))

vet.answer = vet.answer.apply(lambda x : x.replace('\n ', ' '))
vet.answer = vet.answer.apply(lambda x : x.replace(' \n', ' '))
vet.answer = vet.answer.apply(lambda x : x.replace('\n', ''))

vet.answer = vet.answer.apply(lambda x : x.replace('*', ''))
vet.answer = vet.answer.apply(lambda x : x.replace('-', ''))

## 공백 관련 전처리
vet['answer'] = vet['answer'].str.replace(r'([.,?!])(?=[^\s])', r'\1 ', regex=True) ## 문장부호 뒤에 공백 추가하기 (공백이 없는 경우)
vet['answer'] = vet['answer'].str.replace(r'\s([.,?!])', r'\1', regex=True) ## 문장부호 앞에 있는 공백 제거
vet['answer'] = vet['answer'].str.replace(r'\s{2,}', ' ', regex=True) ## 공백이 두 개 이상인 경우 하나의 공백으로 교체
vet['answer'] = vet['answer'].str.strip() ## 문장 양쪽의 공백 제거

## 이상한 문장부호같은거 없는 클린한 답변만 남기기
vet = vet[vet['answer'].str.contains('[a-zA-Z가-힣0-9.,?!()/~ ]')]

#### 3) 이미지가 여러 개 있으면 첫 번째 이미지로 결정

In [7]:
vet.images = vet.images.apply(lambda x : x.split('|')[0])

## 답변 1024 이상과 20 미만인 경우는 제거
vet['answer_length'] = vet.answer.apply(lambda x : len(x))
vet = vet.loc[(vet.answer_length < 1024) & (vet.answer_length >= 20)]

## 3. Preprocessing ( Cat 지식in )

In [9]:
# df = df[['title', 'question', 'images', 'answer']]
df.question = df.question.fillna('랄랄라')
df.images = df.images.drop_duplicates(keep='first')
df = df.dropna()

In [10]:
## 고양이 위주의 글만 추출
df = df[(df.title.str.contains('고양이 피부')) | (df.title.str.contains('고양이 종괴')) | (df.title.str.contains('고양이 여드름')) |
        (df.title.str.contains('고양이 두드러기')) | (df.title.str.contains('고양이 비듬')) | (df.title.str.contains('고양이 각질')) | (df.title.str.contains('반려묘')) |
        (df.question.str.contains('고양이 피부')) | (df.question.str.contains('고양이 종괴')) | (df.question.str.contains('고양이 여드름')) |
        (df.question.str.contains('고양이 두드러기')) | (df.question.str.contains('고양이 비듬')) | (df.question.str.contains('고양이 각질')) | (df.question.str.contains('반려묘'))]
df = df[(~df.title.str.contains('강아지')) & (~df.title.str.contains('알레르기')) & (~df.question.str.contains('알레르기'))]
df = df[~df.title.str.contains('강아지') & (~df.title.str.contains('알레르기')) & (~df.question.str.contains('알레르기')) & ~df.question.str.contains('강아지') & ~df.answer.str.contains('강아지')]

## 이상한 답변 제거
df = df[(~df.answer.str.contains('존재하지 않는 이미지입니다')) & (~df.answer.str.contains('반려견의 피부질환으로 인해')) & (~df.answer.str.contains('다묘 가정의 집사입니다.')) &
   (~df.answer.str.contains('COUPANG')) & (~df.answer.str.contains('naver')) & (~df.answer.str.contains('아토피')) & (~df.answer.str.contains('co.kr')) & (~df.answer.str.contains('bit.ly')) &
   (~df.answer.str.contains('ct'))]

## [질문 요약]이 있는 경우 답변만 사용
df.answer = df.answer.apply(lambda x: ' '.join(x.split(']')[2:]) if '[질문 요약]' in x else x)

## '채택' 키워드가 포함된 경우에 해당 문장 제거
df.answer = df.answer.apply(lambda x: '.'.join([sentence for sentence in x.split('.') if "채택" not in sentence]))
df.answer = df.answer.apply(lambda x: '.'.join([sentence for sentence in x.split('.') if "지식iN" not in sentence]))
df.answer = df.answer.apply(lambda x: '.'.join([sentence for sentence in x.split('.') if "지식인" not in sentence]))
df.answer = df.answer.apply(lambda x: '.'.join([sentence for sentence in x.split('.') if "안녕하세요" not in sentence]))
df.answer = df.answer.apply(lambda x: '.'.join([sentence for sentence in x.split('.') if "감사합니다" not in sentence]))
df.answer = df.answer.apply(lambda x: '.'.join([sentence for sentence in x.split('.') if "위에 기술한" not in sentence]))
df.answer = df.answer.apply(lambda x: '.'.join([sentence for sentence in x.split('.') if "수컷" not in sentence]))
df.answer = df.answer.apply(lambda x: '.'.join([sentence for sentence in x.split('.') if "NAVER" not in sentence]))
df.answer = df.answer.apply(lambda x: '.'.join([sentence for sentence in x.split('.') if "http" not in sentence]))
df.answer = df.answer.apply(lambda x: '.'.join([sentence for sentence in x.split('.') if "1:1" not in sentence]))
df.answer = df.answer.apply(lambda x: '.'.join([sentence for sentence in x.split('.') if ":)" not in sentence]))

## '#'이 있는 경우에 광고성 글이 많아서 다 삭제
df = df[~df.answer.str.contains('#')]
df = df[~df.answer.str.contains('vet/')]
df = df[~df.answer.str.contains('기부')]

## ㅎㅎ, ㅠㅠ, ^^ 와 같은 감정표현 제거
df.answer = df.answer.apply(lambda x : x.replace('ㅎ', ''))
df.answer = df.answer.apply(lambda x : x.replace('ㄷ', ''))
df.answer = df.answer.apply(lambda x : x.replace('^', ''))
df.answer = df.answer.apply(lambda x : x.replace('ㅠ', ''))
df.answer = df.answer.apply(lambda x : x.replace('ㅜ', ''))
df.answer = df.answer.apply(lambda x : x.replace('0', ''))
df.answer = df.answer.apply(lambda x : x.replace('욥', '요'))
df.answer = df.answer.apply(lambda x : x.replace('용', '요'))
df.answer = df.answer.apply(lambda x : x.replace('욘', '요'))
df.answer = df.answer.apply(lambda x : x.replace('[', ' '))
df.answer = df.answer.apply(lambda x : x.replace('~', ''))
df.answer = df.answer.apply(lambda x : x.replace('(? )', ''))

## 오타교정
df.answer = df.answer.apply(lambda x : x.replace('네 ', ''))
df.answer = df.answer.apply(lambda x : x.replace('낫', '났'))
df.answer = df.answer.apply(lambda x : x.replace('너모', '너무'))
df.answer = df.answer.apply(lambda x : x.replace('내요', '내용'))
df.answer = df.answer.apply(lambda x : x.replace('피료', '필요'))
df.answer = df.answer.apply(lambda x : x.replace('을로', '으로'))
df.answer = df.answer.apply(lambda x : x.replace('동냐', '동네'))
df.answer = df.answer.apply(lambda x : x.replace('셧네', '셨네'))
df.answer = df.answer.apply(lambda x : x.replace('부작요', '부작용'))
df.answer = df.answer.apply(lambda x : x.replace('약요샴푸', '약용샴푸'))

## 이상한 문장부호같은거 없는 클린한 답변만 남기기
df = df[df['answer'].str.contains('[a-zA-Z가-힣0-9.,?!() ]')]

## 답변중에 ?가 있는데 length가 짧은 경우 '젖꼭지 아닌가요?'와 같은 답변 column인데 질문하는 요상한 경우가 있어서 ?가 있으면 100자 이상인 경우만 남기기
df = df.drop(df[df['answer'].str.contains('\?')].index)

## 답변중에 그냥 병원에 가보라는 글이 있으니 이러한 경우에 answer_length가 50 아래인 경우 제거
df = df.drop(df[(df['answer'].str.contains('병원')) & (df['answer_length'] < 50)].index)

## 공백 관련 전처리
df['answer'] = df['answer'].str.replace(r'([.,?!])(?=[^\s])', r'\1 ', regex=True) ## 문장부호 뒤에 공백 추가하기 (공백이 없는 경우)
df['answer'] = df['answer'].str.replace(r'\s([.,?!])', r'\1', regex=True) ## 문장부호 앞에 있는 공백 제거
df['answer'] = df['answer'].str.replace(r'\s{2,}', ' ', regex=True) ## 공백이 두 개 이상인 경우 하나의 공백으로 교체
df['answer'] = df['answer'].str.strip() ## 문장 양쪽의 공백 제거

<>:65: SyntaxWarning: invalid escape sequence '\?'
<>:65: SyntaxWarning: invalid escape sequence '\?'
C:\Users\82109\AppData\Local\Temp\ipykernel_2740\3045934677.py:65: SyntaxWarning: invalid escape sequence '\?'
  df = df.drop(df[df['answer'].str.contains('\?')].index)


In [11]:
## 답변 1024 이상과 30 미만인 경우는 제거, 수의사 글이 아니라서 20으로 걸러도 잘 안걸러짐
df['answer_length'] = df.answer.apply(lambda x : len(x))
df = df.loc[(df.answer_length < 1024) & (df.answer_length >= 50)]

In [12]:
df.shape

(1393, 9)

In [13]:
print('라이펫 :', ear.shape)
print('지식인 강아지 :', vet.shape)
print('지식인 고양이 :', df.shape)

라이펫 : (110, 6)
지식인 강아지 : (4175, 7)
지식인 고양이 : (1393, 9)


## 4. Dataset Concat

In [14]:
# 110개의 데이터, 문장 길이는 54 ~ 549
ear.columns = ['category', 'title', 'type', 'question', 'images', 'answer']
ear = ear[['title', 'question', 'images', 'answer']]

In [15]:
vet = vet[['title', 'question', 'images', 'answer']]

In [16]:
df = df[['title', 'question', 'images', 'answer']]

In [17]:
concat_df_0815 = pd.concat([ear, vet, df])

In [18]:
concat_df_0815 = concat_df_0815.sample(frac=1).reset_index(drop=True)

In [19]:
concat_df_0815.to_csv('concat_df_0815.csv', index = False, encoding = 'utf-8')

## 5. Image Labeling

In [ ]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import requests
import pandas as pd
from tqdm.auto import tqdm
import torch  # PyTorch 추가

# 모델과 이미지 프로세서 로드
processor = AutoImageProcessor.from_pretrained("nvidia/mit-b5")
model = AutoModelForImageClassification.from_pretrained("nvidia/mit-b5")

# CUDA 사용 가능 여부 확인 및 모델을 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

In [ ]:
label = pd.read_csv('concat_df_0815.csv')

In [ ]:
# 데이터프레임의 이미지 URL이 포함된 컬럼을 입력받아 라벨을 분류하는 함수
def classify_image(url):
    try:
        # 이미지 다운로드
        response = requests.get(url, stream=True)
        response.raise_for_status()  # 네트워크 오류 처리
        image = Image.open(response.raw)
        
        # 이미지 전처리 및 모델 입력 생성
        inputs = processor(images=image, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}  # 입력 데이터를 GPU로 이동
        
        # 예측 수행
        outputs = model(**inputs)
        logits = outputs.logits
        
        # 예측된 클래스 인덱스
        predicted_class_idx = logits.argmax(-1).item()
        
        # 예측된 라벨 반환
        return model.config.id2label[predicted_class_idx]
    except Exception as e:
        print(f"Error processing image {url}: {e}")
        return "Error"

# tqdm을 사용하여 진행 상황을 시각적으로 표시
tqdm.pandas(desc="Processing Images")
label['label'] = label['images'].progress_apply(classify_image)

In [192]:
## 답변에서 추가 전처리
label = label[~label.answer.str.contains('CT')] ## CT 키워드 포함된 행 제거
label = label[~label.answer.str.contains('Xray')] ## Xray 키워드 포함된 행 제거
label = label[~label.answer.str.contains('탈구')] ## 탈구 키워드 포함된 행 제거
label = label[~label.answer.str.contains('혈뇨')] ## 혈뇨 키워드 포함된 행 제거
label = label[~label.answer.str.contains('유치')] ## 유치 키워드 포함된 행 제거
label = label[~label.answer.str.contains('치석')] ## 유치 키워드 포함된 행 제거
label = label[~label.answer.str.contains('영구치')] ## 영구치 키워드 포함된 행 제거
label = label[~label.answer.str.contains('방사선')] ## 방사선 키워드 포함된 행 제거
label = label[~label.answer.str.contains('너구리')] ## 너구리 키워드 포함된 행 제거
label = label[~label.answer.str.contains('어금니')] ## 어금니 키워드 포함된 행 제거
label = label[~label.answer.str.contains('엑스레이')] ## 엑스레이 키워드 포함된 행 제거
label = label[~label.answer.str.contains('3묘 집사')] ## 3묘 집사 키워드 포함된 행 제거

## 문장 마지막에 괄호가 있다면 제거
label['answer'] = label['answer'].apply(lambda x: x[:-1] if x.endswith(')') else x)

label.answer = label.answer.apply(lambda x: ' '.join([sentence for sentence in x.split(' ') if "은평동물병원장" not in sentence])) ## 은평동물병원장 문장 제거
label.answer = label.answer.apply(lambda x: ' '.join([sentence for sentence in x.split(' ') if "인터파크" not in sentence])) ## 인터파크 문장 제거
label.answer = label.answer.apply(lambda x: ' '.join([sentence for sentence in x.split(' ') if "분양샵" not in sentence])) ## 분양샵 문장 제거

## 라벨이 1개인 경우는 이상한 사진이 올라온 경우가 많아 이런 경우 제거
label_counts = label['label'].value_counts()
label = label[label['label'].map(label_counts) > 1]

## 1개 이상인 경우에 개별 제거
label = label[~label.label.str.contains('crossword puzzle')] ## crossword puzzle
label = label[~label.label.str.contains('tiger shark')] ## tiger shark
label = label[~label.label.str.contains('scoreboard')] ## scoreboard
label = label[~label.label.str.contains('soap dispenser')] ## soap dispenser
label = label[~label.label.str.contains('Carassius auratus')] ## Carassius auratus
label = label[~label.label.str.contains('brassiere')] ## brassiere
label = label[~label.label.str.contains('thimble')] ## thimble
label = label[~label.label.str.contains('walkingstick')] ## walkingstick
label = label[~label.label.str.contains('bird')] ## bird
label = label[~label.label.str.contains('jay')] ## jay
label = label[~label.label.str.contains('African gray')] ## African gray
label = label[~label.label.str.contains('bag')] ## bag
label = label[~label.label.str.contains('rule')] ## rule
label = label[~label.label.str.contains('frypan')] ## frypan
label = label[~label.label.str.contains('honeycomb')] ## honeycomb
label = label[~label.label.str.contains('snail')] ## snail
label = label[~label.label.str.contains('snake')] ## snake
label = label[~label.label.str.contains('plate')] ## plate
label = label[~label.label.str.contains('mousetrap')] ## mousetrap
label = label[~label.label.str.contains('oil filter')] ## oil filter
label = label[~label.label.str.contains('Phalangium opilio')] ## Phalangium opilio
label = label[~label.label.str.contains('lacewing fly')] ## lacewing fly
label = label[~label.label.str.contains('wallet')] ## wallet
label = label[~label.label.str.contains('ballpen')] ## ballpen
label = label[~label.label.str.contains('conch')] ## conch
label = label[~label.label.str.contains('lollipop')] ## lollipop
label = label[~label.label.str.contains('electrical switch')] ## electrical switch
label = label[~label.label.str.contains('hook, claw')] ## hook, claw
label = label[~label.label.str.contains('emmet')] ## emmet
label = label[~label.label.str.contains('oxygen mask')] ## oxygen mask
label = label[~label.label.str.contains('plectrum')] ## plectrum
label = label[~label.label.str.contains('toilet')] ## toilet
label = label[~label.label.str.contains('sunscreen')] ## sunscreen
label = label[~label.label.str.contains('carton')] ## carton
label = label[~label.label.str.contains('eraser')] ## eraser
label = label[~label.label.str.contains('monitor')] ## monitor
label = label[~label.label.str.contains('slug')] ## slug
label = label[~label.label.str.contains('jellyfish')] ## jellyfish
label = label[~label.label.str.contains('CRT screen')] ## CRT screen
label = label[~label.label.str.contains('bottle')] ## bottle
label = label[~label.label.str.contains('Band Aid')] ## Band Aid
label = label[~label.label.str.contains('towel')] ## towel
label = label[~label.label.str.contains('packet')] ## packet
label = label[~label.label.str.contains('menu')] ## menu
label = label[~label.label.str.contains('website')] ## website
label = label[~label.label.str.contains('envelope')] ## envelope
label = label[~label.label.str.contains('Error')] ## Error

## 오타교정
label.answer = label.answer.apply(lambda x : x.replace('비요', '비용')) ## 비요 -> 비용
label.answer = label.answer.apply(lambda x : x.replace('벼우언', '병원')) ## 벼우언 -> 병원
label.answer = label.answer.apply(lambda x : x.replace('가능서', '가능성')) ## 가능서 -> 가능성
label.answer = label.answer.apply(lambda x : x.replace('진도개', '진돗개')) ## 진도개 -> 진돗개

## 추가 세부 전처리
label['answer'] = label['answer'].apply(lambda x: x[:-1] if x.endswith(')') else x) ## 문장 마지막에 괄호가 있다면 제거
label = label[label['answer'].str.contains('[가-힣]')] ## 한글이 없는 경우 제거

In [195]:
label.to_csv('final_df_0815.csv', index = False, encoding = 'utf-8')